In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('diabetes.csv')

In [4]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [6]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [8]:
X = scaler.fit_transform(X)

In [9]:
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2, random_state=1)

In [11]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [12]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=8))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
model.compile(optimizer = 'Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
model.fit(X_train,y_train, batch_size=32, epochs=10, validation_data=(X_test,y_test))

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.4481 - loss: 0.7937 - val_accuracy: 0.5584 - val_loss: 0.7060
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5583 - loss: 0.7120 - val_accuracy: 0.6494 - val_loss: 0.6451
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6799 - loss: 0.6480 - val_accuracy: 0.6818 - val_loss: 0.6002
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7185 - loss: 0.5992 - val_accuracy: 0.7532 - val_loss: 0.5688
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7324 - loss: 0.5712 - val_accuracy: 0.7597 - val_loss: 0.5426
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7421 - loss: 0.5423 - val_accuracy: 0.7792 - val_loss: 0.5232
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7174 - loss: 0.5467 - val_accuracy: 0.8182 - val_loss: 0.5082
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7540 - loss: 0.5138 - val_accuracy: 0.8052 - val_

In [15]:
pip install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.2 MB/s eta 0:00:00


In [16]:
import kerastuner as kt

<ipython-input-16-5fd8096cdee5>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [17]:
def build_model(hp):
    print("Building model with hyperparameters:", hp.values)
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta'])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [18]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='my_tuning_results',
    project_name='binary_classifier'
)


Building model with hyperparameters: {}


In [19]:
tuner = kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=5)

Building model with hyperparameters: {}


In [20]:
tuner.search(X_train,y_train, epochs=5, validation_data=(X_test, y_test))

Trial 4 Complete [00h 00m 08s]
val_accuracy: 0.6623376607894897

Best val_accuracy So Far: 0.7597402334213257
Total elapsed time: 00h 00m 17s


In [21]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [22]:
model = tuner.get_best_models(num_models=1)[0]

Building model with hyperparameters: {'optimizer': 'adam'}


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [23]:
model.summary

<bound method Model.summary of <Sequential name=sequential, built=True>>

In [24]:
model.fit(X_train,y_train, batch_size=32, epochs=10, validation_data=(X_test,y_test))

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7324 - loss: 0.5627 - val_accuracy: 0.7597 - val_loss: 0.5533
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7622 - loss: 0.5107 - val_accuracy: 0.7727 - val_loss: 0.5329
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7579 - loss: 0.5122 - val_accuracy: 0.7727 - val_loss: 0.5189
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7758 - loss: 0.4906 - val_accuracy: 0.7662 - val_loss: 0.5104
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7530 - loss: 0.5031 - val_accuracy: 0.7662 - val_loss: 0.5027
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7869 - loss: 0.4667 - val_accuracy: 0.7597 - val_loss: 0.4978
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7952 - loss: 0.4655 - val_accuracy: 0.7597 - val_loss: 0.4921
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7847 - loss: 0.4618 - val_accuracy: 0.7597 - val_loss

In [25]:
def build_model(hp):
    print("Building model with hyperparameters:", hp.values)
    model = Sequential()
    units = hp.Int('units', min_value=8,max_value = 128,step=8)
    model.add(Dense(units=units, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))
    # optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta'])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [26]:
tuner = kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=5,
                        directory='mydir')

Building model with hyperparameters: {}


In [27]:
tuner.search(X_train,y_train, epochs=5, validation_data=(X_test, y_test), verbose=0)

Building model with hyperparameters: {'units': 32}
Building model with hyperparameters: {'units': 128}
Building model with hyperparameters: {'units': 88}
Building model with hyperparameters: {'units': 8}
Building model with hyperparameters: {'units': 48}


In [28]:
tuner.get_best_hyperparameters()[0].values

{'units': 128}

In [29]:
model = tuner.get_best_models(num_models=1)[0]

Building model with hyperparameters: {'units': 128}


In [30]:
model.fit(X_train,y_train, batch_size=32, epochs=10, validation_data=(X_test,y_test))

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.7394 - loss: 0.4859 - val_accuracy: 0.7987 - val_loss: 0.4723
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7888 - loss: 0.4367 - val_accuracy: 0.7922 - val_loss: 0.4695
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7688 - loss: 0.4621 - val_accuracy: 0.7857 - val_loss: 0.4702
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7933 - loss: 0.4431 - val_accuracy: 0.7662 - val_loss: 0.4727
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7854 - loss: 0.4389 - val_accuracy: 0.7922 - val_loss: 0.4694
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7897 - loss: 0.4384 - val_accuracy: 0.7857 - val_loss: 0.4690
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8051 - loss: 0.4234 - val_accuracy: 0.7857 - val_loss: 0.4712
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7948 - loss: 0.4245 - val_accuracy: 0.7727 - va

In [31]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,845 (15.02 KB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,564 (10.02 KB)

In [39]:
def build_model(hp):
    print("Building model with hyperparameters:", hp.values)
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=8))
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
      model.add(Dense(72, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [40]:
tuner = kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=5,
                      directory='mydir',
                      project_name="layers")

Reloading Tuner from mydir/layers/tuner0.json


In [41]:
tuner.search(X_train,y_train, epochs=5, validation_data=(X_test, y_test), verbose=0)

In [42]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5, 'optimizer': 'rmsprop'}

In [43]:
model = tuner.get_best_models(num_models=1)[0]

Building model with hyperparameters: {'num_layers': 5, 'optimizer': 'rmsprop'}


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 16 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [44]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 72)                  │           9,288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 72)                  │           5,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 72)                  │           5,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 72)                  │           5,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 72)                  │           5,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │              73 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 31,537 (123.19 KB)

 Trainable params: 31,537 (123.19 KB)

 Non-trainable params: 0 (0.00 B)

In [45]:
def build_model(hp):
    print("Building model with hyperparameters:", hp.values)
    model = Sequential()
    counter = 0
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
      if counter == 0:
        model.add(Dense(hp.Int('units'+ str(i), min_value=8, max_value=128, step=8), activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid']), input_dim=8))
      else:
           model.add(Dense(hp.Int('units'+ str(i), min_value=8, max_value=128, step=8), activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid'])))
      counter+=1
    model.add(Dense(1,activation='sigmoid'))
    optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta'])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [46]:
tuner = kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=5,
                      directory='mydir',
                      project_name="final")

Building model with hyperparameters: {}


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [49]:
tuner.search(X_train,y_train, epochs=5, validation_data=(X_test, y_test), verbose=1)

In [50]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 112,
 'activation0': 'tanh',
 'optimizer': 'rmsprop',
 'units1': 8,
 'activation1': 'sigmoid',
 'units2': 88,
 'activation2': 'sigmoid',
 'units3': 88,
 'activation3': 'relu'}

In [51]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 112,
 'activation0': 'tanh',
 'optimizer': 'rmsprop',
 'units1': 8,
 'activation1': 'sigmoid',
 'units2': 88,
 'activation2': 'sigmoid',
 'units3': 88,
 'activation3': 'relu'}

In [52]:
model = tuner.get_best_models(num_models=1)[0]

Building model with hyperparameters: {'num_layers': 1, 'units0': 112, 'activation0': 'tanh', 'optimizer': 'rmsprop', 'units1': 8, 'activation1': 'sigmoid', 'units2': 88, 'activation2': 'sigmoid', 'units3': 88, 'activation3': 'relu'}


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [53]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 112)                 │           1,008 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             113 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,121 (4.38 KB)

 Trainable params: 1,121 (4.38 KB)

 Non-trainable params: 0 (0.00 B)

In [54]:
model.fit(X_train,y_train, batch_size=5, epochs=10, validation_data=(X_test,y_test))

Epoch 1/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7533 - loss: 0.5059 - val_accuracy: 0.7857 - val_loss: 0.4624
Epoch 2/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7686 - loss: 0.5069 - val_accuracy: 0.7922 - val_loss: 0.4621
Epoch 3/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7774 - loss: 0.4883 - val_accuracy: 0.7857 - val_loss: 0.4633
Epoch 4/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7587 - loss: 0.5190 - val_accuracy: 0.7792 - val_loss: 0.4647
Epoch 5/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7631 - loss: 0.4782 - val_accuracy: 0.7857 - val_loss: 0.4685
Epoch 6/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7801 - loss: 0.4778 - val_accuracy: 0.7792 - val_loss: 0.4718
Epoch 7/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7624 - loss: 0.4798 - val_accuracy: 0.7792 - val_loss: 0.4663
Epoch 8/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7730 - loss: 0.4732 - val_accuracy: 0.